In [85]:
import os
import shutil
from ultralytics import YOLO
from pathlib import Path
from PIL import Image, ImageDraw, ImageFont
from tqdm import tqdm

# --- Configuration ---
IMAGE_DIR = 'sample/'
MODEL_PATH = 'models/model-train-test.pt'
VISUAL_OUTPUT_DIR = 'sample_processed/'
ANNOTATION_OUTPUT_DIR = 'yolo_annotations/'

def generate_predictions_for_finetuning():
    """
    Runs the LADaS model, saves raw YOLO .txt files, and manually creates
    visualized images with confidence scores on the labels, using a consistent
    color map for each category.
    """
    print("Starting prediction process...")

    # Ensure output directories exist without deleting them.
    os.makedirs(VISUAL_OUTPUT_DIR, exist_ok=True)
    os.makedirs(ANNOTATION_OUTPUT_DIR, exist_ok=True)
    
    temp_output_dir = Path('./temp_yolo_run')
    if temp_output_dir.exists():
        shutil.rmtree(temp_output_dir)

    print(f"Loading model from: {MODEL_PATH}")
    try:
        model = YOLO(MODEL_PATH)
    except Exception as e:
        print(f"❌ Error loading model: {e}")
        return

    # --- Create a consistent color map based on the model's class names ---
    class_names = model.names
    
    # A curated list of high-contrast colors for better readability with white text.
    colors_list = [
        'blue', 'red', 'green', 'orange', 'purple', 'magenta', 'teal', 
        'brown', 'maroon', 'navy', 'olive', 'indigo', 'crimson',
        'darkgreen', 'darkblue', 'darkorange', 'darkred', 'darkviolet',
        'darkcyan', 'darkmagenta', 'sienna', 'saddlebrown', 'darkslategray',
        'firebrick', 'forestgreen', 'mediumblue', 'midnightblue'
    ]
    
    color_map = {
        class_id: colors_list[class_id % len(colors_list)] 
        for class_id in class_names.keys()
    }
    print("Color map for classes created.")

    print(f"Running predictions on images in: {IMAGE_DIR}")
    try:
        results = model.predict(
            source=IMAGE_DIR,
            save_txt=True,
            project=str(temp_output_dir.parent),
            name=temp_output_dir.name,
            exist_ok=True,
            verbose=False
        )
    except Exception as e:
        print(f"❌ Error during prediction: {e}")
        return

    # --- Manual Visualization Loop ---
    print("Manually drawing bounding boxes with confidence scores...")
    
    for result in tqdm(results, desc="Visualizing Images"):
        try:
            img = Image.open(result.path)
            draw = ImageDraw.Draw(img)
            font_size = 150
            font = ImageFont.truetype("arial.ttf", font_size)
        except IOError:
            font_size = 15 # Fallback if arial.ttf is not found
            font = ImageFont.load_default()
        except Exception as e:
            print(f"Could not open or draw on image {result.path}: {e}")
            continue

        for box in result.boxes:
            class_id = int(box.cls.item())
            class_name = class_names[class_id]
            confidence = box.conf.item()
            xyxy = box.xyxy[0].tolist()
            
            # Use the consistent, high-contrast color from our map
            color = color_map.get(class_id, 'black') # Default to black if not found
            draw.rectangle(xyxy, outline=color, width=3)
            
            label = f"{class_name} {confidence:.2f}"
            
            # Add a background to the text for readability
            text_bbox = draw.textbbox((0, 0), label, font=font)
            text_width = text_bbox[2] - text_bbox[0]
            text_height = text_bbox[3] - text_bbox[1]
            
            text_bg_position = [xyxy[0], xyxy[1] - text_height - 5, xyxy[0] + text_width + 10, xyxy[1]]
            draw.rectangle(text_bg_position, fill=color)
            
            text_position = (xyxy[0] + 5, xyxy[1] - text_height - 5)
            draw.text(text_position, label, fill='white', font=font)
            
        output_image_path = os.path.join(VISUAL_OUTPUT_DIR, os.path.basename(result.path))
        img.save(output_image_path)

    # --- Organize Annotation Files by Overwriting ---
    print("Organizing annotation files...")
    labels_path = temp_output_dir / 'labels'
    if labels_path.exists():
        for txt_file in labels_path.glob('*.txt'):
            destination_path = os.path.join(ANNOTATION_OUTPUT_DIR, txt_file.name)
            os.replace(txt_file, destination_path)
    
    shutil.rmtree(temp_output_dir)
    
    print("\n🎉 Prediction process complete!")
    print(f"✅ Visualized images saved to: {VISUAL_OUTPUT_DIR}")
    print(f"✅ Raw YOLO annotation files saved to: {ANNOTATION_OUTPUT_DIR}")

if __name__ == "__main__":
    generate_predictions_for_finetuning()

Starting prediction process...
Loading model from: models/model-train-test.pt
Color map for classes created.
Running predictions on images in: sample/
Results saved to /Users/carboni/Documents/pynotebook/personal/12.01 Iconographical Canon/Processing/temp_yolo_run
5 labels saved to /Users/carboni/Documents/pynotebook/personal/12.01 Iconographical Canon/Processing/temp_yolo_run/labels
Manually drawing bounding boxes with confidence scores...


Visualizing Images: 100%|██████████| 5/5 [00:00<00:00, 71.23it/s]

Organizing annotation files...

🎉 Prediction process complete!
✅ Visualized images saved to: sample_processed/
✅ Raw YOLO annotation files saved to: yolo_annotations/
